<a href="https://colab.research.google.com/github/nandhukumar86/CapstonePneumoniaDetection/blob/main/Gradio_PneumoniaDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio

  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4714 sha256=50298e6a82ee0d68428c0b35373de9aa6e6830aca177dacbf78a2aa439e13a95
  Stored in directory: c:\users\pvkab\appdata\local\pip\cache\wheels\ff\5b\59\913b443e7369dc04b61f607a746b6f7d83fb65e2e19fcc958d
  Created wheel for flask-cachebuster: filename=Flask_CacheBuster-1.0.0-py3-none-any.whl size=3376 sha256=c7a3ab65b8d499c07d565437badefc0385d794ada8cecc6afac2e46400a64033
  Stored in directory: c:\users\pvkab\appdata\local\pip\cache\wheels\27\1b\de\34a3e06893944387266e2e7e93d33243589f01ac1b451e367b
Successfully built ffmpy flask-cachebuster


You should consider upgrading via the 'c:\users\pvkab\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip install pydicom

You should consider upgrading via the 'c:\users\pvkab\anaconda3\python.exe -m pip install --upgrade pip' command.


In [1]:
import gradio as gr
import pydicom
import io
import tensorflow
import cv2
import numpy as np
from tensorflow.keras import backend as K
from tensorflow.keras.backend import log,epsilon
from tensorflow.keras.models import load_model
from scipy.special import expit

In [2]:
import os, sys
sys.path.append('C:/Learning/GL-AIML/Capstone Project Local')

from colors import get_color
from utils import get_yolo_boxes, makedirs
from bbox import draw_boxes

In [3]:
path = 'C:\Learning\GL-AIML\Capstone Project Local\pneumonia_detection.h5'
model = load_model(path, compile = False)

In [11]:
def detectPneumonia(file):
  f = open(file.name, mode='rb')
  raw_bytes = f.read()
  ds = pydicom.dcmread(io.BytesIO(raw_bytes), force = True)
  f.close();
    
  input_image = cv2.cvtColor(cv2.resize(ds.pixel_array,(256,256)), cv2.COLOR_BAYER_GR2RGB)
  X = []
  X.append(input_image)
  X = np.array(X)

  ###############################
  #   Set some parameters
  ###############################       
  net_h, net_w = 416, 416 # a multiple of 32, the smaller the faster
  obj_thresh, nms_thresh = 0.5, 0.45
  anchors = [55,70, 59,44, 66,100, 76,139, 83,69, 85,188, 99,106, 110,157, 116,214]
  input_path = 'C:/Learning/GL-AIML/Capstone Project Local/input/'
  output_path = 'C:/Learning/GL-AIML/Capstone Project Local/output/'
  image_path = '000db696-cf54-4385-b10b-6b16fbb3f985.JPG'
  image = cv2.imread(input_path + image_path)
  print(image_path)
        
  # predict the bounding boxes
  # boxes = get_yolo_boxes(model, [image], net_h, net_w, config['model']['anchors'], obj_thresh, nms_thresh)[0]
  boxes = get_yolo_boxes(model, [image], net_h, net_w, anchors, obj_thresh, nms_thresh)[0]

  # draw bounding boxes on the image using labels
  draw_boxes(image, boxes, ["opacity"], obj_thresh) 

  # write the image with bounding boxes to file
  cv2.imwrite(output_path+image_path, np.uint8(image))         
            
  output_classify =  model.predict(X)
  output_classify = "Not Identified" if output_classify[0].argmax() == 1 else "Identified"

    
  imagedata = {
      "Patient ID" : ds.PatientID,
      "Age" : ds.PatientAge,
      "Gender" : ds.PatientSex,
      "Body Part" : ds.BodyPartExamined
  }
  return imagedata, ds.pixel_array, output_classify

iface = gr.Interface(fn = detectPneumonia, 
                     inputs = [
                              gr.inputs.File(label="DICOM File")
                     ], 
                     outputs = [
                              gr.outputs.KeyValues(label="Patient Details"),
                              gr.outputs.Image(label="Loaded Image")
                     ],
                     layout = "vertical",
                     title = "Pneumonia Detection",
                     allow_flagging = False)
iface.launch(debug=True)

Running locally at: http://127.0.0.1:7860/
To create a public link, set `share=True` in `launch()`.
Interface loading below...


000db696-cf54-4385-b10b-6b16fbb3f985.JPG
image shape is 256 256
printing boxes in correct_yolo_boxes: <bbox.BoundBox object at 0x000002A84825FDF0>
region= [[ 70  96]
 [ 70  61]
 [153  61]
 [153  96]]


KeyboardInterrupt: 